In [1]:
import pandas as pd
import requests
import json
import base64
import tensorflow as tf

In [2]:
data = pd.read_csv('data/Campus_Clean.csv')
inputs = data.iloc[33, :12].to_dict()
inputs

{'gender': 'F',
 'ssc_p': 87.0,
 'ssc_b': 'Others',
 'hsc_p': 65.0,
 'hsc_b': 'Others',
 'hsc_s': 'Science',
 'degree_p': 81.0,
 'degree_t': 'Comm&Mgmt',
 'workex': 'Yes',
 'etest_p': 88.0,
 'specialisation': 'Mkt&Fin',
 'mba_p': 72.78}

In [3]:
data.iloc[33, :12]

gender                    F
ssc_p                  87.0
ssc_b                Others
hsc_p                  65.0
hsc_b                Others
hsc_s               Science
degree_p               81.0
degree_t          Comm&Mgmt
workex                  Yes
etest_p                88.0
specialisation      Mkt&Fin
mba_p                 72.78
Name: 33, dtype: object

### Mempersiapkan data

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://campus-prediction-production.up.railway.app/v1/models/campus-selection-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Not Accepted" if prediction_value < 0.5 else "Accepted"
else:
    result = "Error: No predictions found in response."
    

Kode ini secara efektif mengintegrasikan fungsi prediksi model machine learning ke dalam alur kerja aplikasi Anda. Dengan beberapa penyesuaian dan perhatian pada detail, Anda dapat memastikan bahwa aplikasi Anda berjalan dengan baik dan memberikan hasil yang akurat berdasarkan input yang diberikan.

### Hasil Predict

In [5]:
print(result)

Accepted
